In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages
!pip install flask flask-ngrok torch transformers diffusers peft ngrok flask_cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
# import flask
# from flask import Flask, request, jsonify, send_file
# from flask_ngrok import run_with_ngrok
# import torch
# from diffusers import DiffusionPipeline
# from peft import PeftModel
# import io
# import uuid
# from huggingface_hub import login

# app = Flask(__name__)
# run_with_ngrok(app)

# # Authentication
# login(token="hf_YourHuggingFaceTokenHere")  # Replace with your Hugging Face token

# # Load base model
# model_id = "black-forest-labs/FLUX.1-dev"
# lora_weights = "AGFirass/Flux1-dev-LoRA"

# # Special loading for FLUX.1-dev
# pipe = DiffusionPipeline.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16,
#     custom_pipeline="stable_diffusion",  # Force SD-compatible loading
#     trust_remote_code=True,  # Required for non-standard models
#     safety_checker=None
# ).to("cuda")

# # Apply LoRA to UNet
# pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_weights)

# @app.route('/generate', methods=['POST'])
# def generate():
#     data = request.json
#     prompt = data['prompt']

#     # FLUX.1-dev requires specific generation settings
#     image = pipe(
#         prompt,
#         num_inference_steps=50,
#         guidance_scale=8.0,  # Recommended for this model
#         width=512,
#         height=512
#     ).images[0]

#     # Return image
#     img_io = io.BytesIO()
#     image.save(img_io, 'PNG')
#     img_io.seek(0)
#     return send_file(img_io, mimetype='image/png')

# if __name__ == '__main__':
#     app.run()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Could not locate the pipeline.py inside stable_diffusion.


EntryNotFoundError: 404 Client Error. (Request ID: Root=1-68199668-36a04a7d786083a65d643c9e;ef560790-eb0a-404a-aee7-b401f160d28f)

Entry Not Found for url: https://huggingface.co/datasets/diffusers/community-pipelines-mirror/resolve/main/v0.33.1/stable_diffusion.py.

In [ ]:
!ls /root/.cache/huggingface/hub/models--black-forest-labs--FLUX.1-dev/snapshots/

0ef5fff789c832c5c7f4e127f94c8b54bbcced44


In [ ]:
!pip show diffusers

Name: diffusers
Version: 0.33.1
Summary: State-of-the-art diffusion in PyTorch and JAX.
Home-page: https://github.com/huggingface/diffusers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/diffusers/graphs/contributors)
Author-email: diffusers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, importlib-metadata, numpy, Pillow, regex, requests, safetensors
Required-by: 


In [ ]:
import flask
from flask import Flask, request, send_file
import torch
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
import io
import uuid
from huggingface_hub import login
import subprocess
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# 1. First install required packages
def install_dependencies():
    subprocess.run(['pip', 'install', 'pyngrok', 'flask', 'torch', 'diffusers', 'accelerate', 'peft'], check=True)

install_dependencies()

# 2. Set up ngrok
from pyngrok import ngrok, conf

# Initialize pipeline
pipe = None

def initialize_pipeline():
    global pipe
    login(token="")
    pipe = FluxPipeline.from_pretrained(
        "black-forest-labs/FLUX.1-dev",
        torch_dtype=torch.bfloat16
    )
    pipe.enable_model_cpu_offload()

    lora_path = "/content/drive/MyDrive/AI/RenamedByFiras/flux1_dev/output/my_first_flux_lora_v1"
    pipe.load_lora_weights(lora_path, weight_name="my_first_flux_lora_v1_000001750.safetensors")

initialize_pipeline()

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    image = pipe(
        data['prompt'],
        height=512,
        width=512,
        guidance_scale=3.5,
        num_inference_steps=50
    ).images[0]

    img_io = io.BytesIO()
    image.save(img_io, 'PNG')
    img_io.seek(0)
    return send_file(img_io, mimetype='image/png')

if __name__ == '__main__':
    # Get your ngrok auth token from https://dashboard.ngrok.com
    ngrok.set_auth_token("")

    # Start ngrok tunnel
    public_url = ngrok.connect(5051).public_url
    print(f" * Running on: {public_url}")

    app.run(host='0.0.0.0', port=5051)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

...

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


 * Running on: https://672b-34-124-179-48.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5051
 * Running on http://172.28.0.2:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:52:14] "OPTIONS /generate HTTP/1.1" 200 -


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [16/May/2025 20:53:03] "POST /generate HTTP/1.1" 200 -
